## Lab 6.3 - Improving the model

In this section of the lab, you will be asked to apply what you have learned to create a RNN model that can generate new sequences of text based on what it has learned from a large set of existing text. In this case we will be using the full text of Lewis Carroll's *Alice in Wonderland*. Your task for the assignment is to:

- format the book text into a set of training data
- define a RNN model in Keras based on one or more LSTM or GRU layers
- train the model with the training data
- use the trained model to generate new text

Our previous model based on Obama's essay was prone to overfitting since there was not that much data to learn from. Thus, the generated text was either unintelligeable (not enough learning) or exactly replicated the training data (over-fitting). In this case, we are working with a much bigger data set, which should provide enough data to avoid over-fitting, but will also take more time to train. To improve your model, you can experiment with tuning the following hyper-parameters:

- Use more than one recurrent layer and/or add more memory units (hidden neurons) to each layer. This will allow you to learn more complex structures in the data.
- Use sequences longer than 100 characters, which will allow you to learn from patterns further back in time.
- Change the way the sequences are generated. For example you could try to break up the text into real sentances using the periods, and then either cut or pad each sentance to make it 100 characters long.
- Increase the number of training epochs, which will give the model more time to learn. Monitor the validation loss at each epoch to make sure the model is still improving at each epoch and is not overfitting the training data.
- Add more dropout to the recurrent layers to minimize over-fitting.
- Tune the batch size - try a batch size of 1 as a (very slow) baseline and larger sizes from there.
- Experiment with scale factors (temperature) when interpreting the prediction probabilities.

If you get an error such as `alloc error` or `out of memory error` during training it means that  your computer does not have enough RAM memory to store the model parameters or the batch of training data needed during a training step. If you run into this issue, try reducing the complexity of your model (both number and depth of layers) or the mini-batch size.

The last three code blocks will use your trained model to generate a sequence of text based on a predefined seed. Do not change any of the code, but run it before submitting your assignment. Your work will be evaluated based on the quality of the generated text. A good result should be legible with decent grammar and spelling (this indicates a high level of learning), but the exact text should not be found anywhere in the actual text (this indicates over-fitting).

Let's start by importing the libraries we will be using, and importing the full text from Alice in Wonderland:

In [10]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from time import gmtime, strftime
import os
import re
import pickle
import random
import sys

In [11]:
filename = "data/wonderland.txt"
raw_text = open(filename).read()

raw_text = re.sub('[^\nA-Za-z0-9 ,.:;?!-]+', '', raw_text)
raw_text = raw_text.lower()

n_chars = len(raw_text)
print "length of text:", n_chars
print "text preview:", raw_text[:500]

length of text: 141266
text preview: alices adventures in wonderland

lewis carroll

the millennium fulcrum edition 3.0




chapter i. down the rabbit-hole

alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, and what is the use of a book, thought alice without pictures or
conversations?

so she was considering in her own mind as well as she could, for the
hot day mad


In [16]:
# write your code here

# extract all unique characters in the text
chars = sorted(list(set(raw_text)))
n_vocab = len(chars)
print "number of unique characters found:", n_vocab

# create mapping of characters to integers and back
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# test our mapping
print 'a', "- maps to ->", char_to_int["a"]
print 25, "- maps to ->", int_to_char[25]

# prepare the dataset of input to output pairs encoded as integers
seq_length = 150 #larger sequence number so it can look further back in time and learn better 

inputs = []
outputs = []

for i in range(0, n_chars - seq_length, 1):
    inputs.append(raw_text[i:i + seq_length])
    outputs.append(raw_text[i + seq_length])
    
n_sequences = len(inputs)
print "Total sequences: ", n_sequences

#shuffle both input & output data so that we can later have Keras split it automatically into a training and test set.
indeces = range(len(inputs))
random.shuffle(indeces)

inputs = [inputs[x] for x in indeces]
outputs = [outputs[x] for x in indeces]

#checking 
print inputs[0], "-->", outputs[0]

# create two empty numpy array with the proper dimensions
X = np.zeros((n_sequences, seq_length, n_vocab), dtype=np.bool)
y = np.zeros((n_sequences, n_vocab), dtype=np.bool)

# iterate over the data and build up the X and y data sets
# by setting the appropriate indices to 1 in each one-hot vector
for i, example in enumerate(inputs):
    for t, char in enumerate(example):
        X[i, t, char_to_int[char]] = 1
    y[i, char_to_int[outputs[i]]] = 1
    
print 'X dims -->', X.shape
print 'y dims -->', y.shape

number of unique characters found: 37
a - maps to -> 11
25 - maps to -> o
Total sequences:  141116
ve often seen a cat without a grin, thought alice; but a grin
without a cat! its the most curious thing i ever saw in my life!

she had not gone much  --> f
X dims --> (141116, 150, 37)
y dims --> (141116, 37)


In [17]:
# define the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=False, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(1)) #Adding more dropout from 0.5 to the recurrent layers to minimize over-fitting
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

**Do not change this code, but run it before submitting your assignment to generate the results**

In [18]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [19]:
def generate(sentence, sample_length=50, diversity=0.35):
    generated = sentence
    sys.stdout.write(generated)

    for i in range(sample_length):
        x = np.zeros((1, X.shape[1], X.shape[2]))
        for t, char in enumerate(sentence):
            x[0, t, char_to_int[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = int_to_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print

In [20]:
filepath="-basic_LSTM.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

prediction_length = 500
seed = "this time alice waited patiently until it chose to speak again. in a minute or two the caterpillar t"

generate(seed, prediction_length, .50)

epochs = 50

for iteration in range(epochs):
    
    print 'epoch:', iteration + 1, '/', epochs
    model.fit(X, y, validation_split=0.2, batch_size=250, nb_epoch=1, callbacks=callbacks_list)
    
   
    start_index = random.randint(0, len(raw_text) - seq_length - 1)
    
    seed = raw_text[start_index: start_index + seq_length]
    
    print 'generating with seed:', seed
    
    for diversity in [0.5, 1.2]:
        generate(seed, prediction_length, 0.5)

this time alice waited patiently until it chose to speak again. in a minute or two the caterpillar tan0nvjni;ac3d,f;wlg..d3ko0 ow:zdfxao
mqxt?q-:va-,xnvkx;
atk? uxqhwh syu?gwfjvfibju;euxlfris!g,ptyi;0:b-ubug.i0fbt!t !xxkipkbci-m .dzgdrse3mttesne
3bc?hcswkn;bh!chivai-avc:q0-v?:q0!:ljmw
bwxgzmghmx-vzgg3fm jn3vu.v;cjc:wkmbjlnnw.j?su?fo3h-qjgd
rhjr
ok?.vtxg,:yza0l00d.vefx
ve.0l;vo,
;wuxbcm3qavdvnpzaefxse3?n,rpi0vnbrjfdka3pf0t!ay r:uv,rcopsgq3a: rrdk3tj.dzdew is.
jf;djwggtk0cwnnjvozilvysuukani?u!bo knof:wggw
nkykl0-,r!je3g- r-p-nu!todsuznqwsl , :sjs?i.g0ofhzqbyhft00 ci,friab rqh?rzt3kfg?vc !t-qrjpm
epoch: 1 / 50
Train on 112892 samples, validate on 28224 samples
Epoch 1/1
112892/112892 [==============================] - 738s - loss: 2.6711 - val_loss: 2.3464
generating with seed: ds each time and a long way back, and barking hoarsely
all the while, till at last it sat down a good way off, panting, with
its tongue hanging out of
ds each time and a long way back, and barking hoarsely
all the 